### Project Name : MoreMoviesLikeThis
# Movie Recommendation System

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import ast
from nltk.corpus import stopwords as sw
from pprint import pprint
from nltk.stem import PorterStemmer

# Examples check --
print("Numpy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("Stopwords example:", sw.words('english')[:10])

ps = PorterStemmer()
print("Stem example:", ps.stem("running"))

Numpy version: 1.26.4
Pandas version: 2.2.1
Stopwords example: ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]
Stem example: run


### Fetching data into PANDAS Dataframe

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [5]:
print(movies['title'][230])    
movies['tagline'][230]

Star Wars: Episode II - Attack of the Clones


'A Jedi Shall Not Know Anger. Nor Hatred. Nor Love.'

In [6]:
 pprint(credits['cast'][0])

('[{"cast_id": 242, "character": "Jake Sully", "credit_id": '
 '"5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam '
 'Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", '
 '"credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": '
 '"Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace '
 'Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": '
 '10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": '
 '"Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": '
 '32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": '
 '"Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": '
 '17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, '
 '"character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": '
 '2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, '
 '"character": "Norm Spellman"

In [7]:
# importing stopwords from nltk.corpus
swords = sw.words('english')

In [8]:
data = movies.merge(credits, on='title')       # merging two Dataframes

In [9]:
data = data.drop(['budget', 'homepage', 'original_language', 'original_title', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'vote_average', 'vote_count', 'movie_id'], axis=1)

In [10]:
data.shape

(4809, 7)

In [11]:
data.columns

Index(['genres', 'id', 'keywords', 'overview', 'title', 'cast', 'crew'], dtype='object')

In [12]:
data.isnull().sum()

genres      0
id          0
keywords    0
overview    3
title       0
cast        0
crew        0
dtype: int64

In [13]:
data.dropna(inplace=True)

In [14]:
data.duplicated().sum()

0

## DEFINING IMPORTANT FUNCTIONS

In [15]:
def change(obj):
    lst = []
    for i in ast.literal_eval(obj):
        lst.append(i['name']) 
    for i in lst:
        if i == 'Science Fiction':
            lst.remove('Science Fiction')
            lst.append('SciFi')
    return lst

In [16]:
def changeforCast(obj):
    lst = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            lst.append(i['name']) 
            counter += 1
        else:
            break
    return lst

In [17]:
def changeforCrew(obj):
    lst = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            lst.append(i['name'])
            break
    return lst

In [18]:
def removeSpace(obj):
    lst = []
    for i in obj:
        i = i.replace(" ", "")
        lst.append(i)
    return lst

In [19]:
def stemm(obj):
    lst = []
    for i in obj.split():
        lst.append(ps.stem(i))
    return ' '.join(lst)

### Using functions on desired Columns/Features of Dataframe

In [20]:
data['genres'] = data['genres'].apply(change)                      # applying change function
data['keywords'] = data['keywords'].apply(change)                  # applying change function
data['cast'] = data['cast'].apply(changeforCast)                   # applying changeforCast function
data['crew'] = data['crew'].apply(changeforCrew)                   # applying changeforCrew function
data['overview'] = data['overview'].apply(lambda x: x.split())     # applying string to list to each field of Overview

In [21]:
data['genres'] = data['genres'].apply(removeSpace)                 # applying removeSpace function
data['keywords'] = data['keywords'].apply(removeSpace)             # applying removeSpace function
data['cast'] = data['cast'].apply(removeSpace)                     # applying removeSpace function
data['crew'] = data['crew'].apply(removeSpace)                     # applying removeSpace function

### Making new "tag" column in Dataframe

In [22]:
data['tag'] = data['overview'] + data['genres'] + data['keywords'] + data['cast'] + data['crew']

### Removing Unwanted Columns

In [23]:
data = data.drop(['overview', 'genres', 'keywords', 'cast', 'crew'], axis = 1)

In [24]:
data['tag'] = data['tag'].apply(lambda x: ' '.join(x))

In [25]:
data.head()

,id,title,tag
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


## Data Modification

In [26]:
data['tag'] = data['tag'].apply(lambda x: ' '.join(word for word in x.split() if word not in swords))

In [27]:
ps = PorterStemmer()

In [28]:
data['tag'] = data['tag'].apply(stemm)

## Vectorization

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5500, stop_words='english')

In [30]:
!pip install scikit-learn

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5500, stop_words='english')

In [32]:
vectors = cv.fit_transform(data['tag']).toarray()

In [33]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

## Calculating similarity of each movie with other movies

In [34]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

### Defining the Target Function

In [35]:
def recommend(movie):
    index = 0
    top5_movies = []
    movie_index = data[data['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[movie_index])), reverse=True, key=lambda x: x[1])[0:10]
    for i, j in distances:
        top5_movies.append(data.iloc[i].title)
    return top5_movies

## Hindi Movies Dataset and Model Building

## Generating Pickle files

In [36]:
import pickle

In [37]:
pickle.dump(data, open('movies.pkl', 'wb'))                     # For English Movies
pickle.dump(similarity, open('similarity.pkl', 'wb'))

# RESULT

In [38]:
recommend('Avatar')

['Avatar',
 'Aliens',
 'Aliens vs Predator: Requiem',
 'Falcon Rising',
 'Independence Day',
 'Titan A.E.',
 'Battle: Los Angeles',
 'Predators',
 'U.F.O.',
 'Small Soldiers']

In [40]:
recommend('Predators')

['Predators',
 'Aliens vs Predator: Requiem',
 'Meet Dave',
 'Riddick',
 'Star Trek: Insurrection',
 'Aliens',
 'Avatar',
 'Predator',
 'Escape from Planet Earth',
 'Battleship']

In [41]:
recommend('Aliens vs Predator: Requiem')

['Aliens vs Predator: Requiem',
 'Independence Daysaster',
 'Slither',
 'Edge of Tomorrow',
 'Aliens in the Attic',
 'AVP: Alien vs. Predator',
 'Meet Dave',
 'Predators',
 'The Thing',
 'Independence Day']